In [1]:
from interest_similarity import interest_similarity
from get_recommended_items import get_recommended_items
from load_data import *
import warnings
warnings.filterwarnings("ignore")
# 노트북 1분 41초

FileNotFoundError: [Errno 2] No such file or directory: 'Data/whyout_data/user_interest.csv'

In [9]:
# user 3은 video만 있음, place, product 없음
# user 20001은 place만 있음, product, video 없음
# user 26232는 product만 있음, place, video 없음 

# 신규 유저는 관심항목 작성
user_id = 7
new_user_item = [1,0,1]
new_user_outdoor = [0,0,0,0,0,0,0,0,0,1]
num_recommendations = 10
case2_dict = { 'place' : [case2_sgd_rating_place, place, case2_user_place, case2_user_place_idx, case2_user_latent_place, case2_item_latent_place, user_place],
         'video' : [case2_sgd_rating_video, video, case2_user_video, case2_user_video_idx, case2_user_latent_video, case2_item_latent_video, user_video],
         'product' : [case2_sgd_rating_product, product, case2_user_product, case2_user_product_idx, case2_user_latent_product, case2_item_latent_product, user_product]}
item = 'product'
item_list = list(case2_dict.keys())

# Case 2만 구현
# if : 기존 유저, else : 신규 유저 구분하기
if user_id in user_interest['idx'].values:
    recomm_list = get_recommended_items(user_id, item, item_list, case2_dict, num_recommendations)
else:
    new_user_id = find_similar_user_interest(item, case2_dict, user_interest, new_user_item, new_user_outdoor)
    recomm_list = get_recommended_items(new_user_id, item, item_list, case2_dict, num_recommendations)
print(f"user {user_id}에게 추천해줄 {num_recommendations}개 {item} idx : {recomm_list}")

user 7에게 추천해줄 10개 product idx : [6089, 2741, 501, 4604, 2842, 3540, 1218, 3511, 1795, 4942]
